In [ ]:
# 从零实现
import torch
from torch import nn
from d2l import torch as d2l


def dropout_layer(X, dropout):
    assert 0 <= dropout <= 1
    # 在本情况中，所有元素都被丢弃
    if dropout == 1:
        return torch.zeros_like(X)
    # 在本情况中,所有元素都被保留
    if dropout == 0:
        return X
    # mask 是一个0或者1的张量，为0的概率就是dropout的概率
    mask = (torch.rand(X.shape) > dropout).float()  # rand:[0-1] 随机分布； randn:标准正态分布
    # mask部位值为1对应位置的元素扩大（1-p）倍，最终X与X"的期望相同
    return mask * X / (1.0 - dropout)  # 这里用 mask*X 是应为做乘法比选元素赋值来的快，有GPU加速


# 定义模型参数
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256


# 靠近输入层的地方设置较低的暂退概率
# 一般就三种取值 0.1 0.5 0.9
# 例如隐藏层是64感觉不错，那就可以改隐藏层是128，dropout为0.5，感觉好像等效 64=128*0.5
# 应为丢掉一半，可能效果更好？
dropout1, dropout2 = 0.2, 0.5

# 定义模型
class Net(nn.Module):  # 继承nn.Module
    def __init__(
        self, num_inputs, num_outputs, num_hiddens1, num_hiddens2, is_training=True
    ):
        super(Net, self).__init__()
        self.num_inputs = num_inputs
        self.training = is_training
        # nn.Linear()自动对参数进行了初始化，是(-\sqrt(k),\sqrt(k))的均匀分布，其中k是输入维度的倒数
        self.lin1 = nn.Linear(num_inputs, num_hiddens1)  # 隐藏层1
        self.lin2 = nn.Linear(num_hiddens1, num_hiddens2)  # 隐藏层2
        self.lin3 = nn.Linear(num_hiddens2, num_outputs)  # 输出层
        self.relu = nn.ReLU()

    # nn.module里的__call__方法里面调用到了forward
    def forward(self, X):
        H1 = self.relu(self.lin1(X.reshape((-1, self.num_inputs))))  # 这里展平了输入
        # 只有在训练模型时才使用dropout 将暂退法应用于每个隐藏层的输出（在激活函数之后）
        # 在计算验证集前，加一句 net.eval()，就会自动关闭dropdout
        if self.training == True:
            # 在第一个全连接层之后添加一个dropout层
            H1 = dropout_layer(H1, dropout1)
        H2 = self.relu(self.lin2(H1))
        if self.training == True:
            # 在第二个全连接层之后添加一个dropout层
            H2 = dropout_layer(H2, dropout2)
        out = self.lin3(H2)
        return out


net = Net(num_inputs, num_outputs, num_hiddens1, num_hiddens2)



In [ ]:
# 训练和测试
num_epochs, lr, batch_size = 10, 0.5, 256
loss = nn.CrossEntropyLoss()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
trainer = torch.optim.SGD(net.parameters(), lr=lr)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)


# 通常模型可以复杂一点，然后通过正则化来控制我们的模型复杂度
# 有dropout的情况下，可以把隐藏层稍微设大一点，dropout的率也设大一点
# 这样可能比不用dropout，隐藏层小 效果好一些
